In [3]:
from heapq import heappop, heappush

# Parse the maze from the input string
def parse_maze(input_str):
    maze = [list(line) for line in input_str.strip().split("\n")]
    start, end = None, None
    for y, row in enumerate(maze):
        for x, cell in enumerate(row):
            if cell == 'S':
                start = (x, y)
            elif cell == 'E':
                end = (x, y)
    return maze, start, end

# Solve the maze using Dijkstra's algorithm
def solve_maze(input_str):
    maze, start, end = parse_maze(input_str)
    
    # Define movement directions: (dx, dy, direction name)
    directions = [(0, -1, 'N'), (1, 0, 'E'), (0, 1, 'S'), (-1, 0, 'W')]
    direction_map = {d[2]: i for i, d in enumerate(directions)}

    # Priority queue for Dijkstra's
    pq = []
    heappush(pq, (0, start[0], start[1], 'E'))  # (cost, x, y, facing)

    # Visited set: (x, y, facing)
    visited = set()
    cost_map = {}  # To track the minimum cost for each tile

    while pq:
        cost, x, y, facing = heappop(pq)

        # If reached the end, return the cost
        if (x, y) == end:
            return cost, cost_map

        # Skip if already visited
        if (x, y, facing) in visited:
            continue
        visited.add((x, y, facing))

        # Store the minimum cost for the tile
        cost_map[(x, y)] = cost

        # Current direction index
        current_dir_index = direction_map[facing]

        # Explore neighbors
        for i, (dx, dy, new_dir) in enumerate(directions):
            nx, ny = x + dx, y + dy

            # Check boundaries before proceeding
            if not (0 <= nx < len(maze[0]) and 0 <= ny < len(maze)):
                continue

            # If moving forward
            if i == current_dir_index:
                if maze[ny][nx] != '#':  # Valid forward move
                    heappush(pq, (cost + 1, nx, ny, new_dir))

            # If turning (rotating clockwise or counterclockwise)
            else:
                turn_cost = 1000
                heappush(pq, (cost + turn_cost, x, y, new_dir))

    return float('inf'), cost_map  # No solution found

# Find all the tiles that are part of at least one of the best paths
def find_best_path_tiles(input_str):
    # Solve the maze to get the minimum cost and cost map
    maze, start, end = parse_maze(input_str)
    min_cost, cost_map = solve_maze(input_str)

    # Set to store all the tiles part of the best paths
    best_path_tiles = set()

    # Check all the tiles to see if they are part of a best path
    for (x, y), cost in cost_map.items():
        # If this tile is part of the shortest path (cost == min_cost)
        if cost == min_cost:
            best_path_tiles.add((x, y))

            # Check neighbors: the tile can be part of the best path if it leads to
            # a neighboring tile with the correct cost
            for dx, dy in [(0, -1), (1, 0), (0, 1), (-1, 0)]:
                nx, ny = x + dx, y + dy
                if (nx, ny) in cost_map and cost_map[(nx, ny)] == cost - 1:
                    best_path_tiles.add((x, y))
    
    # Return the number of best path tiles
    return len(best_path_tiles)

# Read input from file
with open("input.txt", "r") as file:
    input_str = file.read()

# Solve for the number of tiles on the best path
result = find_best_path_tiles(input_str)
print("Number of tiles part of the best paths:", result)


Number of tiles part of the best paths: 2


In [1]:
from collections import deque
from typing import List, Set, Tuple

class MazeSolver:
    def __init__(self, maze: List[List[str]]):
        self.maze = maze
        self.height = len(maze)
        self.width = len(maze[0])
        self.start, self.end = self._find_start_end()

    def _find_start_end(self) -> Tuple[Tuple[int, int], Tuple[int, int]]:
        start = end = None
        for y in range(self.height):
            for x in range(self.width):
                if self.maze[y][x] == 'S':
                    start = (y, x)
                elif self.maze[y][x] == 'E':
                    end = (y, x)
        return start, end

    def find_optimal_tiles(self) -> int:
        # Directions: right, down, left, up
        directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
        
        # First pass: Find minimum score to end from each position and direction
        best_scores = {}  # (y, x, dir) -> min_score
        queue = deque([(self.start[0], self.start[1], 0, 0)])  # y, x, dir, score
        min_end_score = float('inf')
        
        while queue:
            y, x, dir, score = queue.popleft()
            
            if score >= min_end_score:
                continue
                
            state = (y, x, dir)
            if state in best_scores and best_scores[state] <= score:
                continue
            best_scores[state] = score
            
            # Found end
            if (y, x) == self.end:
                min_end_score = min(min_end_score, score)
                continue
            
            # Try moving forward
            ny, nx = y + directions[dir][0], x + directions[dir][1]
            if (0 <= ny < self.height and 0 <= nx < self.width and 
                self.maze[ny][nx] != '#'):
                queue.append((ny, nx, dir, score + 1))
            
            # Try turning
            queue.append((y, x, (dir - 1) % 4, score + 1000))
            queue.append((y, x, (dir + 1) % 4, score + 1000))
        
        # Second pass: Find tiles that are part of optimal paths
        optimal_tiles = set()
        visited = set()
        queue = deque([(self.start[0], self.start[1], 0, 0, {(self.start[0], self.start[1])})])
        
        while queue:
            y, x, dir, score, path = queue.popleft()
            
            if score > min_end_score:
                continue
                
            state = (y, x, dir)
            if score > best_scores.get(state, float('inf')):
                continue
                
            if (y, x) == self.end and score == min_end_score:
                optimal_tiles.update(path)
                continue
            
            # Try moving forward
            ny, nx = y + directions[dir][0], x + directions[dir][1]
            if (0 <= ny < self.height and 0 <= nx < self.width and 
                self.maze[ny][nx] != '#'):
                new_path = path | {(ny, nx)}
                queue.append((ny, nx, dir, score + 1, new_path))
            
            # Try turning
            queue.append((y, x, (dir - 1) % 4, score + 1000, path.copy()))
            queue.append((y, x, (dir + 1) % 4, score + 1000, path.copy()))
        
        return len(optimal_tiles)

def visualize_path(maze: List[List[str]], optimal_tiles: Set[Tuple[int, int]]) -> None:
    for y in range(len(maze)):
        for x in range(len(maze[0])):
            if (y, x) in optimal_tiles:
                if maze[y][x] in ['S', 'E']:
                    print(maze[y][x], end='')
                else:
                    print('O', end='')
            else:
                print(maze[y][x], end='')
        print()

def main():
    sample_maze = """#############################################################################################################################################
#...#.......#.............#.........#...................#.............#.....#.....#...........#.....................................#......E#
#.###.###.###.#####.#######.#######.#.#.#####.#########.#####.###.#.#.###.#.###.#.#####.#####.#.###########.###.#.#.###.###.#######.#.#####.#
#...#.#.#.#...#...#.#.......#.....#.#.#.#.....#.......#.#...........#...#.#.....#.#.....#.#...#.....#.....#.#.....................#.#...#.#.#
#.#.#.#.#.#.###.#.#.#.#######.###.#.#.#.###.###.#####.#.#.#.#.###.#####.#.#######.#.#####.#.#########.###.###.#######.#.#.###.###.#.###.#.#.#
#.#.#...#...#...#.#.......#...#.#.#.#.#...#...#...#...#...#.#.........#...#.....#...#...#.#...#.......#.#.....#.......#...#.#.#...#...#...#.#
#.#.###.#####.#.###########.###.#.#.#.###.#######.#.###.#####.###.###.#.###.###.#####.#.#.###.#.#######.#######.#.###.#####.#.#.###.#.###.#.#
#.#.....#.....#.......#.....#...#.#.#.#...#.....#.#.#...#.....#.....#...#.........#...#...#...#...#...#.........#...#.......#.#.....#...#.#.#
#.#.#############.#####.#######.#.#.#.#.###.#.###.#.#.###.###.#.#.#.###.#.#####.#.#.#####.#.#####.#.#.#.###########.#######.#.#####.###.###.#
#.#.#.......#...#.....#.....#.....#...#.....#.....#.#...#.#.....#.....#.#.#.....#...#...#...#...#.#.#.#.#...#.......#...#.....#...#...#.....#
#.#.#.###.#.#.#.#####.#####.#.###.#.#.#############.#####.###.#.#.###.#.###.###.###.#.#.#.###.#.#.#.#.#.#.#.#.#####.#.#.#.#####.#.#.#.#####.#
#.............#.......#.........#.......#...#.....#.....#.....#.#.#...#...#.#.....#.#.#...#...#.#...#.#.#.#.#.#.....#.#...#.....#...#.....#.#
#.#.###.#######.#######.#######.###.###.#.#.###.#######.#####.###.#.###.#.#.#.#.###.#.###.#.#.#.#.###.#.#.#.#.#.#####.#############.#####.#.#
#...#...#...#.......#...#...#...#.#.#.#...#...#.......#.....#.#...#.....#.#.#.#.....#.#...#...........#...#.#...............#.....#.#.....#.#
###.#.###.#.#.#.###.#.###.#.#.#.#.#.#.#######.###.#.###.#.###.#.#######.#.#.#.#.#####.#.#.###.#.#.#.#######.#.#.###.#######.#.###.#.#.###.#.#
#.#...#...#...#...#.#.#...#...#...#.#...#...#.#...#.#...#.#...#.#.....#...#.#.....#...#.#.....#.#.#.#.....#.#.#...............#.....#.#...#.#
#.#.#.#.#######.###.#.#.###########.#.#.#.#.#.#.###.#.#####.###.#.#.#.#.###.#.#####.###.#######.#.#.#.#####.#.#######.#.###########.#.#####.#
#...#.#.#.....#.......#...#.....#...#.#.#.#...#.#...#...#...#.....#.#.#...#.#.#...#.....#.#.....#.#.#.......#.......#.#.#...#.....#.#.#.....#
#.#.#.#.#.###.#########.#.#.###.#.###.###.#####.#######.#.###.###.#.#.###.#.###.#.#######.#.###.###.#.#############.###.#.#.#.###.#.#.#.#####
#...#.#...#.#.......#...#...#.#...#.....#.....#.......#...#.......#.#...#.#.#...#.........#.#.......#.#...........#...#...#.#.#.......#.#...#
###.#.#####.#######.#####.###.#########.#####.#####.#.###########.#.#.###.#.#.#########.###.#.#.#####.#.#########.###.#####.#.#########.#.#.#
#...#.#...........#.....#...#.#...................#.#...............#.#...#.#.#.......#.....#.#.#.....#...#.....#...#.....#.#...........#.#.#
#.###.###.###.###.#####.###.#.#.###.#.###########.#.###.#######.#####.#.###.#.#.#########.#####.#.#######.#.#######.###.#.#.###############.#
#...#...#...#...#...#.#...#.#...#...#.#.........#.#.#...#.....#.....#.#...#.....#.......#...#...#...#...#.#.............#...#.......#.....#.#
#.#.###.#######.###.#.###.#.###.#.#####.###.###.#.###.#####.#.#####.#.###.#.###.#.#####.###.#.#######.#.#.###########.###.###.###.###.#.#.#.#
#.#...#...#.....#.#.#...#.#...#.#.........#...#.#.....#.....#...#...#...#.....#...#.#...#.#...#.......#...#...#.....#.#...#...#...#...#.#.#.#
#.#.#####.#.#####.#.###.#.###.#.###########.#.#.#######.###.###.#.###.###.###.#.#.#.#.###.#######.#######.#.#.#.###.###.###.#.#.###.###.#.#.#
#.#.#.....#.#.#.........#.#.#.#.........#...#.....#.....#...#.#.#.#.......#...#.#.#.....#.........#.......#.#...#.#.#...#.#.#.........#.#...#
###.#.###.#.#.#.#########.#.#.#########.#.#.###.#.#.#####.#.#.#.#.#####.#.#.###.#.#.###.#######.###########.#####.#.#.###.#.###.###.#.#.###.#
#...#...#...#.....#.......#.#.....#...#.#.......#.#.#...#.#.#...#.........#...#.#...#.#.......#.....#.....#.#.#...#...#.....#.......#.#...#.#
#.#.###.#.#.#####.#.#######.#####.#.###.#######.#.#.#.#.#.#.###.#########.###.#####.#.#######.#.###.#.###.#.#.#.#.#########.#.#.###.#.###.#.#
#.#.......#...#.#.#.#...........#.#...#.#...#...#.....#...#...#.#.......#...#...........#.....#...#...#.#...#.#.#...........#...#...#...#.#.#
#.###.###.###.#.#.#.#.###.#.#####.#.#.#.###.#.#####.#.#####.#.#.#####.#.###.###########.#.#######.#####.#####.#.#.#######.#.###.#.#######.###
#...#.#.#...#.#.#.#.....#.#.......#.#...#...#.#.....#.#.....#.#.......#.#...#.....#...#.#.#.#.....#.........#...#...#...#.#.#.#.#.......#...#
###.#.#.#.#.#.#.#.#####.#.#####.#######.#.#.#.###.###.###.#.#.###########.###.#.###.#.###.#.#.#######.#.#####.#####.###.#.#.#.#.#######.#.#.#
#.............#.#.....#.......#.#.....#.#.#.#...#...#.....#.#.....#...#...#...#.#...#.....#.#.......#.#...#...#...#.....#.#.#...............#
#.###.#.#.#.#.#.#####.#######.#.#.###.#.#.#.###.#####.#######.###.#.#.#.###.###.#.#########.#######.###.#.#.###.#.#.#.#.#.#.#.#.#.#.#.#####.#
#.#...#.....#.#...#...#.#.....#...#...#.......#.......#.........#...#...#...#...#.#.......#...#...#...#.#...#...#.#.#.#.#.#...#...#.#.......#
#.#####.#.###.###.#.###.#.#########.#.#######.#########.###.#.#########.#.###.#.#.#.#.#.#.#.###.#.###.#.#####.###.#.#.###.#.###.###.#######.#
#.....#.......#...#.....#...#...................#...#...#...#...#.....#...#.....#.#.#...#...#...#...#...#...#.....#.#.....#.#...#.......#...#
#.###.#.#.#####.###.#######.#####.###########.#.#.###.###.###.#.#.###.#######.###.###.#####.#.#####.#.#.#.#.#.#.###.#####.###.#####.#.#.#.###
#.#.....#.....#.....#.....#.....#.#.........#.....#...#.#.#...#...#.#.......#...#...#.....#.#.#...#...#...#...#...#.........#.....#.#.#.#.#.#
#.#.###.#.#.#.#.#####.###.#.###.#.#.#.#####.#######.###.#.#.#######.#######.#.#####.#.###.###.###.#############.#######.###.#####.###.###.#.#
#.#.......#.....#.#...#...#.#...#.#.#.....#.......#.#.......#.......#.....#.#.#.....#.#.#.....#.....#...........#.......#.......#.#...#...#.#
#.###.#.#.#######.#.#####.###.###.#######.#.#####.#.###########.#.###.#.###.###.#####.#.#######.###.#.#.#########.#.###.#######.#.#.###.###.#
#...#...#...........#...#...#.#.#.........#.#...#.#...........#.#.....#.......#.#.....#...#.......#...#.......#...#...#.#.....#...#.#.......#
###.###.#.###.###.#.#.#.###.#.#.#####.#####.#.#.#.#########.#.#.#############.#.#.#####.#.#.#####.###########.#.###.#.#.#.###.#####.#.#######
#.#.....#...#.#...#...#...#...#.....#.#.......#.#.#...........#.#.....#...#...#.#.#.....#.#.#...#.......#.....#...#.#.#.#...#.#.......#.....#
#.###.#.###.#.#.#########.#####.###.#.#.#######.#.#.#######.###.#.#.###.#.#.###.#.#.###.###.#.#.#######.#.#####.#.#.#.#.###.#.#########.###.#
#.#.....#...#.#.......#...#...#.#...#.#.#...#...#...#.........#.....#...#.#.....#.#...#.....#.#...#.....#.#.....#.#.#.#.#.#.#.........#.#.#.#
#.#.#.#.#.###.###.#.###.#.#.#.###.###.#.#.#.#.#######.#######.#####.#.###.#######.#######.#.#.#.#.#.#####.###.###.#.#.#.#.#.#########.#.#.#.#
#...#.....#.....#.......#...#...#.....#.#.#.#...#.....#.....#...#...#...#...#.............#.#.#.#...#...#...#.#...#.#.#.#.#...#...#...#.#.#.#
#.###.#.###.#.###.###.#####.###.#.#####.###.###.###.#####.#####.#.#####.###.###########.#.#.#.#.###.#.#.###.#.#.#####.#.#.###.#.###.###.#.#.#
#.#...#.#.#.#.#...#.#.......#...#.....#.......#.........#.#...#.#...#.#.#.............#.#.#.....#...#...#...#.#.#.....#.....#.#.#...#...#...#
#.###.#.#.#.#.#.###.#########.###.#####.#####.###.###.#.#.#.#.#.###.#.#.###########.#.#.#.###.###.###.#.#.#####.#.###.#####.#.#.#.###.###.#.#
#.......#...#.#...#.....#.#...#...#.....#.......#...#.#.#...#.#...#.#.#.....#.......#.#.#...#.#...#.#.#.........#...#.#.....#.#.......#.#...#
#####.#####.#.###.#.###.#.#.#######.#####.#.#######.###.#.###.###.#.#.#####.###.#####.#.###.###.###.#.###.#######.#.###.#####.#########.###.#
#.....#...#.#...#.#.#.....#...#...#.#.....#.#.....#.#...#.#.....#.#...#.....#...#...#.#.....#...#.....#.........#.#...#.....#.....#.....#...#
#.###.#.#.#.#.#.#.#.#########.#.#.#.#.#####.#.###.#.#.#####.###.#.###.#.#####.###.#.#.#.#####.###.#####.#######.#####.#####.#####.#.###.#.###
#.#.....#.#...#.#.#...#.....#...#.......#.#...#.#...#.....#.....#.#.#.#.....#...#.....#.....#.#.....#...#.......#.........#...#...#.#.#...#.#
#.###.###.###.#.#.###.#.###.#.#########.#.#####.#.###.###.#####.#.#.#.#####.#.#.#.#########.#.#.#####.###.#######.#.#########.#.###.#.#####.#
#.#...#.........#.....#.#...#...#...#...#.....#...#...#...#...#.....#.....#.#.#...#...#...#...#.#.....#...#.#.....#.#.......................#
#.#.#.#########.###.###.#.#####.#.#.#.###.###.###.#.###.#.#.#.###.#######.#.###.###.#.#.#.#######.#####.###.#.###.###.#########.#.#.#.#####.#
#.............#...#.#...#.....#...#.#.#.....#.....#.#...#.#.#...#.#...#...#...#.......#.#.........#...#...#.#...#.#.........#...#.#...#.#...#
#.#.#.#######.###.#.#.###.###.#.#.#.#.#####.###.###.#.#.#.#.###.###.#.#.#####.#######.#.#.#.#######.#.###.#.###.#.#.###.###.#.###.#####.#.###
#.......#...#...#.#.#.......#.....#...#...#.#...#...#.#.#...#.#.....#...#...#.#.....#...#...#.......#.#...#.....#.......#...#...#.....#...#.#
#####.#.#.#.###.###.#######.###.#######.#.#.###.#.###.#.#.###.#.#.#.#####.###.#.###.#########.#.#.#.###.#############.#########.#.#####.###.#
#.........#...#.#...#.#.....#...#...#...#.#...#.#.#...#.#.....#.#.#.........#.#...#.........#.......#...#.......#...#.#...#.....#...#...#...#
#.#.#######.###.#.###.#.#.#######.#.#.###.#.#.#.#.#.#.#######.#.#.#########.#.###.###.#####.###.#.###.###.#####.#.#.###.#.#.#####.#.#.#####.#
#.#.......#.#...#.#...#.#.........#...#.#.#.#.#...#.#.........#.#...#.......#...#.#...#...#.....#...#.#.......#.........#.#...#...#.#.......#
#.#######.###.#.#.#.#.#.###.#########.#.#.###.###.#.###.#.#####.###.#.###.#.###.#.#####.#.#########.#.#######.###########.#.#.#.###.#######.#
#.#.#.........#.#.#.#.#...#.....#...#...#...........#...#.#...#...#...#.#...#...#.......#.....#.....#.#.......#.........#.....#...#.....#.#.#
#.#.#.#.#########.###.###.#.#####.#.#######.#.#######.#.###.#.#.#.###.#.#####.###.#######.###.#.#####.#.#######.#######.###########.###.#.#.#
#.#.#.#.....#.....#...#.#...#...#...#.......#.#...#...#.....#.#.#.....#.#.....#.#...#.......#.#.#.....#...#.....#...#.......#.......#.....#.#
#.#.#.#.###.#.#####.#.#.#####.#.#.#.#.###.#####.#.#.#########.#####.###.#.#####.###.#####.#.#.#.#.#######.#.#####.#.#.#####.#.#.#####.#####.#
#.#.#.#.............#...#.....#...#.#.#...#.....#.#...#.....#.....#.#.#.#.....#.#...#.....#.#.#.#...#.....#.#...#.....#.....#.#...#.#...#...#
#.#.#.#.#.#####.###.#####.#########.#.#.###.#####.###.#.#########.#.#.#.#####.#.#.###.#####.#.#.###.#.#####.#.###.#####.#########.#.###.#.###
#.#...#.......#...#.#...........#.....#.#...#...#...#.#.#.......#.#...#.#.....#.......#.....#.#.#...#.#.....#.#...#...............#.....#...#
#.#####.#####.#.###.#.###.#.###.#########.###.###.#.#.#.#.###.#.#.###.#.#.###################.###.###.#.#####.#.#############.#############.#
#...........#.#.#...#.#...#...........#.......#...#.#.#.....#.#.#...#.#...#.......................#...#.......#.#.....#.#.....#...........#.#
#.#####.###.#.###.###.#######.#######.#.#######.#####.#######.###.#.#.#####.#.#############.#.#.#####.#######.#.#.###.#.#.#####.###.###.###.#
#...#.....#.#...#.#...#.......#.....#...#...#...#...#.........#.....#.#...#.#.#.......#...#...#.....#.#.#.....#.#.#.#.#...#...#...#.........#
###.#.#.###.###.#.#.###.#########.#######.#.#.###.#.###.#####.#.#####.#.#.#.#.###.###.#.###.#######.#.#.#.#####.#.#.#.#.#.#.#.###.#.#.#######
#.#...#.#...#.#.#.#.....#...#.....#.......#...#...#.....#.......#...#.#.#.#.#...#.#.....#...#.......#.#.......#...#.#.#...#.#.#...#.#...#...#
#.###.###.#.#.#.#.#######.#.###.#.#.#####.###.#.#######.#.#######.#.#.#.#.#.###.#.#######.###.#.#.#.#.#.#####.#####.#.###.#.#.#.#####.#.###.#
#.....#...#...#.....#.....#...#.#.#.#.....#.........#...#.#.......#...#.#...#...#...........#.#.#.#...#.#...#.#...#.#...#.#.#.#.#.....#...#.#
#.#####.###########.#.#######.#.#.#.#.#####.#####.###.#.#.#####.#########.###.###.###########.#.###.#.###.#.#.#.#.#.###.#.#.###.###.#####.#.#
#...#.#.#.......#...#.#.........#.#.#.#...#.........#.#...#...#.........#...#.#.#...#.........#.#...#.....#.#...#...#.#.#...#...#...#.......#
###.#.#.#.###.###.###.#########.###.###.#.###.#.#.#.#.###.#.#.#########.#####.#.#.###.#########.#.###.#####.#######.#.#.#####.###.###########
#...#.#...#.#.....#.........#...#...#...#.....#...#.#...#.#.#.#.......#.......#...#...#.......#...#.#.#.....#.....#...#.......#.........#...#
#.###.#####.###############.#.###.#.#.###########.#.#.#.###.#.#.#####.#####.#####.#.###.###.#######.#.#.#####.###.###.###########.#####.#.#.#
#.....................#.....#.#...#.#.#.........#.#.#.#.#...#.#.....#.#...#.#...#.#.#.....#.......#...#.............#...........#...#.#...#.#
#######.#############.#.#####.#.#.###.#.#####.#.#.#.###.#.###.#####.#.#.#.###.#.#.#.#########.#.#.#############.###.###########.###.#.#####.#
#.............#.....#.#.....#.#.#.#...#.#...#.#.#.#...#.#...#.....#.#...#...#.#.#...........#.#.#...........#.....#.#.......#.#...#.......#.#
#.#.###.#####.###.###.#####.#.#.###.###.#.#.#.###.###.#.###.#.#.###.#######.#.#.###########.###.###########.###.#.#.#####.#.#.###.#######.#.#
#.#.........#.....#...#.....#.#.....#.....#...#...#...#.#...#.#.#.......#.#.#.#.....#.....#.#...#...#.....#...#...#.#.....#.#...#.#.......#.#
#.#####.###.#####.#.#.#.#####.#.#.###.#####.###.#####.#.#.###.###.#####.#.#.#.###.#.#.###.#.#.###.#.#.###.###.#####.#.#####.###.#.#######.#.#
#...#...........#.#.#...#...#.#.......#.....#...#...#...#.#.#.....#...#...#.......#.#...#.#.#...#.#.#...#...#.......#.#...#.....#...#.....#.#
###.#.#.#.#.#.#.#.#.#.###.###.###.#.#########.###.#.###.#.#.#######.#.#########.#.#.#.###.#.###.#.#.#####.#.#########.#.#.#####.###.#.###.#.#
#...#.#...#.#.#...#.#.#.#...#...#.#...#.......#...#.#...#.....#...#.#.......#.#.#.#.#.#...#...#...#.#.........#.......#.#.....#.#.#...#...#.#
#.#.#.#.#.#.#####.#.#.#.#.#.###.#####.#.#####.#.###.#########.###.#.#######.#.#.#.###.#.#####.###.#.#.#######.###.#.###.#####.#.#.#####.#.#.#
#.#...#...#.....#...#.....#...#.....#.#...#...#.#...#.......#.....#...#.....#.........#.....#...#.#.#.......#.#.....#.#.#.#...#.#.....#.#...#
#.#.#.#.#######.#.###.#####.#####.#.#.###.#####.#.###.#####.#.#######.#.###################.###.#.#.#####.#.#.#.#####.#.#.#.###.###.#.#.#.###
#.....#...#...#...#.............#.#.#...#...#...#.....#...#...#.......#...#.......#.......#...#.#.#...#...#.#...#.........#.............#...#
###.#####.#.#.#####.###.#.###.#.#.#.#.#.###.#.#########.#######.#####.###.#.#####.#.#####.#.#.#.#####.#.###.#####.#######.###.#.#.###.#.#.#.#
#.......#...#.....#.#...#.....#...#.#.#...#...#...............#...#.#.......#...#...#.....#.#.#.#.....#...#.....#.........#.............#.#.#
###.#############.#.#.#.#########.#.###.#.#####.#.#####.#.#######.#.#.#########.#####.#.###.###.#.#####.#.###.#.###.#######.#######.#.#.#.#.#
#...#...........#...#.#.#...#.....#.....#...#...#.......#.#.......#.....#.....#...#...#...#...#.#.....#.#...#.#...#.#.#.....#.........#.....#
#.#.#.#####.#########.###.#.###.###########.#.###########.#.#.#######.###.###.#.###.#.###.###.#.#.###.#.###.#.###.#.#.#.#.###.#######.###.#.#
#.#.......................#...#...#...#.#...#...#.........#.#.#.....#.#.....#.#.....#...#.....#.......#...#.#.#.#.#.#.#.#.#.....#...#.....#.#
#.#.#.#.#.#################.#.###.#.#.#.#.###.###.#########.###.###.###.###.#.###.#####.#####.#############.#.#.#.#.#.#.#.#.#.###.#.#####.#.#
#.#.#...#.#.......#.....#.#...#.#...#.#.#.#.......#.....#...#...#.#.#...#...#...#.....#.......#.............#.#.#.#...#.#.#.#...#.#.........#
#.#######.#.#####.#.#.#.#.#.###.#####.#.#.#####.#.#.###.#.#.#.###.#.#.#####.###.#####.#############.#########.#.#.#####.#.#.#.#.#.#######.###
#.......#...#...#.#.#.#...#.#.......#.#.#.......#.#.#.#.#.#.#...#.#...#.....#...#...#.......#.......#.#.......#.#.......#.#.#.#...#.........#
#.###.#######.###.#.#.###.#.#######.#.#.#####.#####.#.#.#.#.###.#.#####.#####.###.#.#######.#.#######.#.#.#####.#########.#.#.#####.#####.#.#
#.#.#...........#.#.#...#.#.#.....#.....#...#.#.....#.....#...#.....#.......#.#...#.........#.#...#.....#...#.....#.......#.#.#.....#...#...#
#.#.#######.###.#.#####.#.#.#.###.#####.#.###.#.#######.#####.#####.#######.#.#.#.###########.#.#.#########.#.#.###.#######.#.#.###.#.#.#.###
#...#.#...#...#.#.......#.#...#.#.....#...#...#.........#.........#.#...#...#.#...............#.#.........#.#.#.#.......#...#...#...#.#...#.#
###.#.#.#.#####.###.#.###.###.#.#####.#####.#####.#####.#.#########.#.#.#.###.#####.#####.#####.#########.#.#.#.#.#######.#######.#.#.#####.#
#.....#.#.....#...#...#...#.....#...#.....#.#.....#...#.#.#.....#...#.#.#...#...#...#...#.#...#.#.....#.....#.#...#.....#.......#.#...#.....#
#######.#####.#.#.#.#.###.###.###.#.#####.#.###.###.#.#.#.#.###.#.###.#.###.###.#####.#.###.#.#.#.#.#.###.#########.###.#####.#.#.#.#####.#.#
#.......#...#.#...#.#...#...#.....#.....#.#.....#...#.....#.....#...#.#...#...#.......#.....#.#.....#...#.#...........#.#...#...............#
#.#########.#.#.#.#.###.###.#####.###.#.#.#####.#.#######.#.#.#.###.#.###.#.#######.#########.#.#######.###.#########.#.#.#.#.#.#.#.#.#.###.#
#.............#.#.....#...#.#...#...#.#...#.....#.....#...#.#.#.#...#...#...#.....#.....#...#.....#...#...#.#...#...#.#.#.#...#.#.....#.#...#
#.#.#.#.#.#.#.#.#####.###.#.#.#.#.#.###.#.#.#########.#.###.#.###.#####.###.#.###.#####.#.#######.#.#.###.#.#.###.#.###.#.#####.###.###.#.###
#.#.#...#.#.#.#.#.....#...#...#...#.....#...#.......#.#...#.#.#.........#...#.#.#.#...#.#...#.....#.#...#.........#...#.......#...#...#.#...#
#.#######.###.#.#.#####.#######.#####.#.###.#.#####.#.###.#.#.#.#############.#.#.#.#.#.###.#.#####.###.#.###.#.#####.#######.#.#.#.###.###.#
#.......#.......#.#.....#...........#.......#...#.#...#.....#.#...#...........#.......................#.......#.#...#.......#...#.#.#...#...#
#.#####.#########.#.###.#.#.###.###.#.#####.###.#.###.#.#.###.###.#.###.###.#########.#########.#.###.###.###.#.#.#.#######.#####.#.#.###.###
#.#...#...#...#...#.#.#...............#.........#.#...#...#.#...#.#.....#...#.....................#...#.....#...#.#.#.....#.....#...#.#.#...#
#.###.###.#.#.#.###.#.#.#.#.#.###.###.#.###.#####.#.#####.#.#.###.#####.#.###.###.#############.#.#.#.#.#.#.#.###.#.###.#.#.###.#.#.#.#.#.#.#
#.#.....#...#.#.#.#...#.#.#.#.#.#.....#.........#.#.....#...#...#.....#.#...#.#.#.............#.#.................#.....#...#...#.#...#...#.#
#.#.###.#####.#.#.###.#.###.#.#.###########.###.#.#####.#.#.###.#####.#.#####.#.#############.#.#.#.#.###.###.#.#####.#####.#.###.#.#######.#
#...#.#.....#...#...#.#...#.#.#.........#.......#.....#.#.#...#.#.....#.#.....#.....#...#.....#.#.#.#...#.#...#...#...#.....#.#...#.....#...#
#####.#.###.#####.#.#.###.#.#.###.#.###.#.#.#####.#.###.#.###.#.#.#####.#.#####.###.#.#.#.#.#.###.#.#####.###.###.#.#.#####.#.#####.###.#.#.#
#...#.......#.....#.....#.#.#...#.#...#.#...#.....#.#.....#...........................#...#.#.....#...#.........#...#.....#.#.#.....#...#...#
#.#.#.###################.#.###.#####.#.###.#.#####.#.###.#.#######.###.#.#####.#.#########.#######.#.#.#.###.#####.#####.#.#.#.###.#.###.#.#
#.#...#...........#...#...#...#...#...#...#.#.#...#...#...#...#.#...#...#.#...#.#.........#.......#.........#.#...#.....#.#.#.#...#.#...#.#.#
#.#######.#######.#.#.#.###.#.###.#.#####.#.###.#.#####.#.###.#.#.#####.#.#.#.#.#.#.#############.#######.#.###.#.#######.#.#.###.#.###.###.#
#S........#.........#...#...#.......#.....#.....#.......#.......#...........#...#.#.......................#.....#.........#.#.....#.........#
#############################################################################################################################################
"""
    maze = [list(line) for line in sample_maze.splitlines()]
    solver = MazeSolver(maze)
    result = solver.find_optimal_tiles()
    print(f"Number of tiles in optimal paths: {result}")

if __name__ == "__main__":
    main()

Number of tiles in optimal paths: 529
